In [2]:
# Start Fresh
 
%reset -f
! pip install emoji
! pip install catboost
import csv  # for slang
import os
import re  # regex
import string  # punct
from pprint import pprint
 
import emoji  # for emoji
import gensim
import keras
import lightgbm as lgb
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
import seaborn as sns
import sklearn
from gensim.models import Word2Vec
from IPython.display import Image
from matplotlib import pyplot as plt
from nltk.corpus import stopwords  # stopwords
from nltk.stem import PorterStemmer  # stemming
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn import svm, tree
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, RandomForestRegressor,
                              StackingClassifier)
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import (accuracy_score, auc, average_precision_score,
                             brier_score_loss, classification_report,
                             confusion_matrix, f1_score, fbeta_score,
                             make_scorer, plot_precision_recall_curve,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (GridSearchCV, KFold, RandomizedSearchCV,
                                     cross_val_score, train_test_split)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler
from sklearn.svm import SVC  # "Support vector classifier"
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
 
import catboost as cb
import xgboost as xgb
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform
 
# pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import nltk
nltk.download('stopwords')
from operator import itemgetter

 
%matplotlib inline
from IPython.display import clear_output
 
from google.colab import files # to read slang.txt until better solution is foun
uploaded = files.upload()
import time
start_time = time.time()
clear_output()

In [3]:
# Clean Text Class

class CleanText(BaseEstimator, TransformerMixin):
    
    def remove_mentions(self, input_text):
        '''
        Remove mentions, like @Mplamplampla
        '''
        return re.sub(r'@+', '', input_text)
    
    def remove_urls(self, input_text):
        '''
        Remove the urls mention in a tweet
        '''
        input_text  = ' '.join([w for w in input_text.split(' ') if '.com' not in w])
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        input_text = emoji.demojize(input_text)
        input_text = input_text.replace('_','')
        input_text = input_text.replace(':','')
        return input_text
    
    def possessive_pronouns(self, input_text):
        '''
        Remove the possesive pronouns, because otherwise after tokenization we will end up with a word and an s
        Example: government's --> ["government", "s"]
        '''
        return input_text.replace("'s", "")
    
    def characters(self, input_text):
        '''
        Remove special and redundant characters that may appear on a tweet and that don't really help in our analysis
        '''
        input_text = input_text.replace("\r", " ") # Carriage Return
        input_text = input_text.replace("\n", " ") # Newline
        input_text = " ".join(input_text.split()) # Double space
        input_text = input_text.replace('"', '') # Quotes
        return input_text
    
    def remove_punctuation(self, input_text):
        '''
        Remove punctuation and specifically these symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        '''
        punct = string.punctuation # string with all the punctuation symbols '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    
    def remove_digits(self, input_text):
        '''
        Remove numbers
        '''
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        '''
        Convert all the sentences(words) to lowercase
        '''
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        '''
        Remove stopwords (refers to the most common words in a language)
        '''
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        '''
        Reduce the words to their stem
        '''
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def encode_decode(self, input_text):
        '''
        Remove weird characters that are result of encoding problems
        '''
        return  " ".join([k.encode("ascii", "ignore").decode() for k in input_text.split(" ")])
    
    
    def translator(self, input_text):
        '''
        Transform abbrevations to normal words
        Example: asap --> as soon as possible
        '''
        input_text = input_text.split(" ")
        j = 0
        for _str in input_text:
            # File path which consists of Abbreviations.
            fileName = "/content/slang.txt"
            # File Access mode [Read Mode]
            accessMode = "r"
            with open(fileName, accessMode) as myCSVfile:
                # Reading file as CSV with delimiter as "=", so that 
                # abbreviation are stored in row[0] and phrases in row[1]
                dataFromFile = csv.reader(myCSVfile, delimiter="=")
                # Removing Special Characters.
                _str = re.sub('[^a-zA-Z0-9-_.]', '', _str)
                for row in dataFromFile:
                    # Check if selected word matches short forms[LHS] in text file.
                    if _str.upper() == row[0]:
                        # If match found replace it with its appropriate phrase in text file.
                        input_text[j] = row[1]
                myCSVfile.close()
            j = j + 1
        
        return(' '.join(input_text))
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords)
                    .apply(self.stemming))
        return clean_X
    
    def transform_no_stem(self, X, **transform_params):
        clean_X = (X.apply(self.translator)
                    .apply(self.remove_mentions)
                    .apply(self.remove_urls)
                    .apply(self.emoji_oneword)
                    .apply(self.possessive_pronouns)
                    .apply(self.remove_punctuation)
                    .apply(self.remove_digits)
                    .apply(self.encode_decode)
                    .apply(self.characters)
                    .apply(self.to_lower)
                    .apply(self.remove_stopwords))
        return clean_X

In [5]:
our_text = "I just wanted to say that @Aladdin you've been an inspiration for my webpage http://www.iloveAladdin.com. Also i ❤️ Jasmin's beautiful 24K  ring. \nNot to complain but i need one asap  Ã¥"
print(our_text)

I just wanted to say that @Aladdin you've been an inspiration for my webpage http://www.iloveAladdin.com. Also i ❤️ Jasmin's beautiful 24K  ring. 
Not to complain but i need one asap  Ã¥


In [6]:
ct = CleanText()

#Abbreviations

In [7]:
our_text = ct.translator(our_text)
print(our_text) # asap ---> As Soon As Possible

I just wanted to say that @Aladdin you've been an inspiration for my webpage http://www.iloveAladdin.com. Also i ❤️ Jasmin's beautiful 24K  ring. 
Not to complain but i need one As Soon As Possible  Ã¥


# Remove Mentions

In [8]:
our_text = ct.remove_mentions(our_text)
print(our_text) # @Aladdin ---> Aladdin

I just wanted to say that Aladdin you've been an inspiration for my webpage http://www.iloveAladdin.com. Also i ❤️ Jasmin's beautiful 24K  ring. 
Not to complain but i need one As Soon As Possible  Ã¥


# Remove URLs

In [9]:
our_text = ct.remove_urls(our_text)
print(our_text) # http://www.iloveAladdin.com. ---> ____

I just wanted to say that Aladdin you've been an inspiration for my webpage Also i ❤️ Jasmin's beautiful 24K  ring. 
Not to complain but i need one As Soon As Possible  Ã¥


# Emoji Decoder

In [10]:
our_text = ct.emoji_oneword(our_text)
print(our_text) # ❤️ ---> redheart

I just wanted to say that Aladdin you've been an inspiration for my webpage Also i redheart Jasmin's beautiful 24K  ring. 
Not to complain but i need one As Soon As Possible  Ã¥


# Remove Possessive Pronouns

In [11]:
our_text = ct.possessive_pronouns(our_text)
print(our_text) # Jasmine's ---> Jasmine

I just wanted to say that Aladdin you've been an inspiration for my webpage Also i redheart Jasmin beautiful 24K  ring. 
Not to complain but i need one As Soon As Possible  Ã¥


# Remove Punctuation

In [12]:
our_text = ct.remove_punctuation(our_text)
print(our_text) # [', .] ---> ____

I just wanted to say that Aladdin you ve been an inspiration for my webpage Also i redheart Jasmin beautiful 24K  ring  
Not to complain but i need one As Soon As Possible  Ã¥


# Remove Digits

In [13]:
our_text = ct.remove_digits(our_text)
print(our_text) # 24K ---> K

I just wanted to say that Aladdin you ve been an inspiration for my webpage Also i redheart Jasmin beautiful K  ring  
Not to complain but i need one As Soon As Possible  Ã¥


# Remove Weird Characters 

In [14]:
our_text = ct.encode_decode(our_text)
print(our_text) # Ã¥ ---> ___

I just wanted to say that Aladdin you ve been an inspiration for my webpage Also i redheart Jasmin beautiful K  ring  
Not to complain but i need one As Soon As Possible  


# Remove characters like tab, next line double space etc

In [15]:
our_text = ct.characters(our_text)
print(our_text) # \n ---> ___

I just wanted to say that Aladdin you ve been an inspiration for my webpage Also i redheart Jasmin beautiful K ring Not to complain but i need one As Soon As Possible


# All the capital to lowercase

In [16]:
our_text = ct.to_lower(our_text)
print(our_text) # \n ---> ___

i just wanted to say that aladdin you ve been an inspiration for my webpage also i redheart jasmin beautiful k ring not to complain but i need one as soon as possible


# Remove stopwords

In [17]:
our_text = ct.remove_stopwords(our_text)
print(our_text) # [i, just, to, that, you, ve, been, an, for, my, k, but, i, as] ---> ___

wanted say aladdin inspiration webpage also redheart jasmin beautiful ring not complain need one soon possible
